# Exercício AV03

Com base no arquivo que representa um recorte dos alvarás de curitiba, anexo à atividade.

1. Abra o arquivo, e liste os comandos utilizados abaixo:

In [219]:
#Resposta
dtf <- read.table("https://raw.githubusercontent.com/leonardodecuritiba/rdatascience/master/aula03/base_alvaras.csv",header=T,sep=";",dec=".")
head(dtf)

NOME_EMPRESARIAL,INICIO_ATIVIDADE,DATA_EXPIRACAO,BAIRRO
<fct>,<fct>,<fct>,<fct>
"(MPDS) MEIO AMBIENTE, PESSOAS E O DESENVOLVIMENTO",04/02/2014,,CACHOEIRA
(RBDS) NO RASTRO DA BIODIVERSIDADE E DO DESENVOLVI,14/12/2012,,CIDADE INDUSTRIAL
+55 ADMINISTRADORA DE BENS LTDA.,24/03/2016,,SAO LOURENCO
-SUELI DE FATIMA OLIVEIRA-,19/08/2019,,CIDADE INDUSTRIAL
0 STELLA SERVICOS E COMERCIO DE CARIMBOS,24/04/2003,,BUTIATUVINHA
0 SILVA & ANTUNES LTDA,01/08/2001,,FANNY


2. Defina uma nova coluna contendo a quantidade de anos, da empresa. Considere que o campo DATA_EXPIRACAO vazio, representa que a empresa ainda existe:

In [0]:
install.packages("lubridate")
library(lubridate)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


In [0]:
#Resposta
#Primeiro precisamos alterar o formato das datas para o padrão aceito pela biblioteca lubridate

#Exemplos
d2 <- '2015-01-02'
ymd(d2) #usando formato ano-mes-dia
d1 <- '01-02-2015'
dmy(d1)  #usando formato dia-mes-ano

#para isso, podemos usar o comando gsub, e substituir a coluna do dataframe original
dtf$INICIO_ATIVIDADE = gsub("/", "-", dtf$INICIO_ATIVIDADE)
dtf$DATA_EXPIRACAO = gsub("/", "-", dtf$DATA_EXPIRACAO)
head(dtf)


In [0]:
#A coluna dtf$DATA_EXPIRACAO não possui alguns valores, logo podemos substituir os valores vazios pela data de hoje

#obtendo a data atual
ano_atual <- format(Sys.Date(), "%d-%m-%Y")
#obtendo os índices vazios
index <- dtf$DATA_EXPIRACAO == ""
dtf$DATA_EXPIRACAO[index] <- ano_atual

head(dtf)

In [0]:
vetor_data_inicio <- dmy(dtf$INICIO_ATIVIDADE) #vetor contendo as datas
head(vetor_data_inicio)

In [0]:
vetor_ano_inicio <- year(vetor_data_inicio) #definindo vetor contendo apenas os anos
head(vetor_ano_inicio)

In [0]:
dtf$ANO_INICIO <- vetor_ano_inicio
dtf$ANO_EXPIRACAO <- year(dmy(dtf$DATA_EXPIRACAO))
head(dtf)

In [0]:
dtf$IDADE <- dtf$ANO_EXPIRACAO - dtf$ANO_INICIO #calculando a idade
head(dtf)

3. Apresente o histograma para esta distribuição de anos de vida:

In [0]:
#Resposta
range(dtf$IDADE)
hist(dtf$IDADE,nclass=10,xlab="Idade das Empresas",ylab="Frequencia",main="Histograma")


4. Apresente o gráfico em formato de pizza.


In [0]:
#Resposta
#Primeiro podemos agrupar por idade, e soma da idade
agrupado = aggregate(dtf$IDADE,
          by = list(dtf$IDADE),
          FUN=sum)
head(agrupado)


In [0]:
#Agora vamos ordenar pelo número de anos
colnames(agrupado) <- c("anos","frequencia") #renomeando as colunas
reordenado <- agrupado[order(agrupado$anos, decreasing = FALSE),]
head(reordenado)

In [0]:
#Pegando os 6 primeiros valores
a= reordenado$frequencia[1:6] #pegando as frequencias
b = reordenado[,1] #pegando os anos
pie(a, b)

5. Com relação à simetria dos dados, o que você pode afirmar?


In [0]:
#Resposta
# a distribuição das frequências possui assimetria à direita
# é do tipo Despinhadeiro
# A maior parte das empresas tem um tempo de 5 anos


6. (Desafio Extra) Agrupe as empresas por Bairro e crie uma coluna com a quantia de empresas naquele bairro.

In [0]:
#Resposta
#Para instruções mais complexas podemos utilizar a biblioteca dplyr
library(tidyverse)

In [0]:
tdtf <- as_tibble(dtf) #transformando um dataframe, convertendo-o em um tibble dataframe (tbl_df)
head(tdtf)

In [0]:
tdtf %>%
  group_by(BAIRRO) %>% #agrupando por numero de quartos e número de vagas
  summarise(count=n()) #aplicando contagem de empresas naquele bairro

7. (Desafio Extra) Liste os 5 bairros com maior índice de empresas fechadas, utilizando um gráfico do tipo pizza. Responda, qual bairro possui mais empresas fechadas?

In [0]:
#uma possibilidade seria listar as empresas cujo ano de expiração é diferente de 2019, ou seja, pela lógica, ainda não fecharam
fechadas <- tdtf[tdtf$ANO_EXPIRACAO < 2019,]
fechadas

In [0]:
#Agora agrupamos por bairro e aplicamos a contagem nesse subset
fechadas_c <- fechadas %>%
  group_by(BAIRRO) %>% #agrupando por numero de quartos e número de vagas
  summarise(count=n()) %>% #aplicando contagem de empresas naquele bairro
  arrange(desc(count)) #ordenando por count
fechadas_c

In [0]:
fechadas_c[1:5,] #selecionando os 5 bairros com empresas mais fechadas